In [1]:
%cd mmdetection

# !pip install -q --upgrade wandb
# !pip3 install openmim
# !mim install mmcv-full
# !pip install -e .

/aiffel/aiffel/aimmo/mmdetection


In [2]:
from mmdet.apis import init_detector, inference_detector, show_result_pyplot
import mmcv
from mmcv import Config
from mmdet.datasets import build_dataset
from mmdet.models import build_detector
from mmdet.apis import train_detector
from mmdet.datasets.builder import DATASETS
from mmdet.datasets.coco import CocoDataset,CustomDataset
from mmdet.apis import set_random_seed
import os.path as osp
import os
import numpy as np

import wandb
print(wandb.__version__)

/opt/conda/lib/python3.9/site-packages/mmcv/__init__.py:20: UserWarning: On January 1, 2023, MMCV will release v2.0.0, in which it will remove components related to the training process and add a data transformation module. In addition, it will rename the package names mmcv to mmcv-lite and mmcv-full to mmcv. See https://github.com/open-mmlab/mmcv/blob/master/docs/en/compatibility.md for more details.
  warnings.warn(


0.13.5


In [3]:
# # 완디비 로그인
wandb.login(key= '')

wandb: Currently logged in as: wodms. Use `wandb login --relogin` to force relogin
wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: Appending key for api.wandb.ai to your netrc file: /aiffel/.netrc


True

In [4]:
wandb.init(project="r-cnn", entity="aimmo")

wandb: Currently logged in as: wodms (aimmo). Use `wandb login --relogin` to force relogin


In [5]:
@DATASETS.register_module(force=True)
class Aimmo(CustomDataset):
    CLASSES = ('car', 'truck', 'bus','pedestrian')
    def load_annotations(self, ann_file):
        cat2label = {k:i for i, k in enumerate(self.CLASSES)}
        image_list = mmcv.list_from_file(self.ann_file)
        data_infos = []
    
        for image_id in image_list:
            filename = os.path.splitext(image_id)[0]#os.path.splitext('{0:}/{1:}'.format(img_prefix, image_id))[0]
            #image = cv2.imread(filename)
            data_info = {'filename': filename,
                         'width': 1920, 
                         'height': 1024}
            label_prefix = self.img_prefix.replace('image', 'middle_texts')
            lines = mmcv.list_from_file(osp.join(label_prefix, str(image_id)))

            content = [line.strip().split(' ') for line in lines]
            bbox_names = [x[0] for x in content]
            bboxes = [ [float(info) for info in x[1:5]] for x in content]

            gt_bboxes = []
            gt_labels = []
            gt_bboxes_ignore = []
            gt_labels_ignore = []

            for bbox_name, bbox in zip(bbox_names, bboxes):
                if bbox_name in cat2label:
                    gt_bboxes.append(bbox)
                    gt_labels.append(cat2label[bbox_name])
                else:
                    gt_bboxes_ignore.append(bbox)
                    gt_labels_ignore.append(-1)
            
            data_anno = {
              'bboxes': np.array(gt_bboxes, dtype=np.float32).reshape(-1, 4),
              'labels': np.array(gt_labels, dtype=np.compat.long),
              'bboxes_ignore': np.array(gt_bboxes_ignore, dtype=np.float32).reshape(-1, 4),
              'labels_ignore': np.array(gt_labels_ignore, dtype=np.compat.long)
            }
            data_info.update(ann=data_anno)
            data_infos.append(data_info)
        return data_infos

In [6]:
config_file = 'configs/faster_rcnn/faster_rcnn_r50_fpn_1x_coco.py'
checkpoint_file = 'checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'
cfg = Config.fromfile(config_file)

cfg.dataset_type = 'Aimmo'
cfg.data_root = 'data/dataset/'

cfg.data.train.type = 'Aimmo'
cfg.data.train.data_root = 'data/dataset/'
cfg.data.train.ann_file = 'middle_train.txt'
cfg.data.train.img_prefix = 'image'


cfg.data.val.type = 'Aimmo'
cfg.data.val.data_root = 'data/dataset/'
cfg.data.val.ann_file = 'middle_val.txt'
cfg.data.val.img_prefix = 'image'


cfg.data.test.type = 'Aimmo'
cfg.data.test.data_root = 'data/dataset/'
cfg.data.test.ann_file = 'middle_test.txt'
cfg.data.test.img_prefix = 'image'

cfg.model.roi_head.bbox_head.num_classes = 4

cfg.load_from = 'checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth'
cfg.work_dir = 'middle'

cfg.lr_config.policy='step'
cfg.seed = 0
set_random_seed(0, deterministic=False)
cfg.gpu_ids = range(1)
cfg.device='cuda'
cfg.evaluation.metric='mAP'

cfg.log_config.interval = 100
cfg.evaluation.interval = 1
# We can set the checkpoint saving interval to reduce the storage cost
cfg.checkpoint_config.interval = 1
#cfg.evaluation.save_best='auto'
cfg.data.train.pipeline[2].img_scale=[(960,512)]#['Resize']
cfg.train_pipeline[2].img_scale=[(960,512)]
cfg.test_pipeline[1].img_scale=[(960,512)]
cfg.data.val.pipeline[1].img_scale = [(960,512)]
cfg.data.test.pipeline[1].img_scale = [(960,512)]
cfg.runner.max_epochs = 50

cfg.data.samples_per_gpu = 2 # Batch size of a single GPU used in testing
cfg.data.workers_per_gpu = 0 # Worker to pre-fetch data for each single GPU
print(cfg.pretty_text)

model = dict(
    type='FasterRCNN',
    backbone=dict(
        type='ResNet',
        depth=50,
        num_stages=4,
        out_indices=(0, 1, 2, 3),
        frozen_stages=1,
        norm_cfg=dict(type='BN', requires_grad=True),
        norm_eval=True,
        style='pytorch',
        init_cfg=dict(type='Pretrained', checkpoint='torchvision://resnet50')),
    neck=dict(
        type='FPN',
        in_channels=[256, 512, 1024, 2048],
        out_channels=256,
        num_outs=5),
    rpn_head=dict(
        type='RPNHead',
        in_channels=256,
        feat_channels=256,
        anchor_generator=dict(
            type='AnchorGenerator',
            scales=[8],
            ratios=[0.5, 1.0, 2.0],
            strides=[4, 8, 16, 32, 64]),
        bbox_coder=dict(
            type='DeltaXYWHBBoxCoder',
            target_means=[0.0, 0.0, 0.0, 0.0],
            target_stds=[1.0, 1.0, 1.0, 1.0]),
        loss_cls=dict(
            type='CrossEntropyLoss', use_sigmoid=True, loss_weight=1.

In [7]:
cfg.log_config.hooks = [
    dict(type='TextLoggerHook'),
    dict(type='WandbLoggerHook',
         init_kwargs={'project': 'r-cnn', 'entity' : 'aimmo', 'name' : 'middle'},
         interval=10,
         log_checkpoint=True,
         log_checkpoint_metadata=True,
         num_eval_images=100)] 

In [8]:
[build_dataset(cfg.data.train)]

/aiffel/aiffel/aimmo/mmdetection/mmdet/datasets/custom.py:181: UserWarning: CustomDataset does not support filtering empty gt images.
  warnings.warn(


[
 Aimmo Train dataset with number of images 8000, and instance counts: 
 +----------+-------+-----------+-------+----------+-------+----------------+-------+---------------+-------+
 | category | count | category  | count | category | count | category       | count | category      | count |
 +----------+-------+-----------+-------+----------+-------+----------------+-------+---------------+-------+
 | 0 [car]  | 44328 | 1 [truck] | 10360 | 2 [bus]  | 6097  | 3 [pedestrian] | 10916 | -1 background | 3     |
 +----------+-------+-----------+-------+----------+-------+----------------+-------+---------------+-------+]

In [9]:
# Build dataset
datasets = [build_dataset(cfg.data.train)]

# Build the detector
model = build_detector(cfg.model, train_cfg=cfg.get('train_cfg'), test_cfg=cfg.get('test_cfg'))
# Add an attribute for visualization convenience
model.CLASSES = datasets[0].CLASSES

# Create work_dir
mmcv.mkdir_or_exist(osp.abspath(cfg.work_dir))
train_detector(model, datasets, cfg, distributed=False, validate=True)

2022-12-06 04:05:06,020 - mmdet - INFO - Automatic scaling of learning rate (LR) has been disabled.
2022-12-06 04:05:06,876 - mmdet - INFO - load checkpoint from local path: checkpoints/faster_rcnn_r50_fpn_1x_coco_20200130-047c8118.pth
2022-12-06 04:05:08,287 - mmdet - WARNING - The model and loaded state dict do not match exactly

size mismatch for roi_head.bbox_head.fc_cls.weight: copying a param with shape torch.Size([81, 1024]) from checkpoint, the shape in current model is torch.Size([5, 1024]).
size mismatch for roi_head.bbox_head.fc_cls.bias: copying a param with shape torch.Size([81]) from checkpoint, the shape in current model is torch.Size([5]).
size mismatch for roi_head.bbox_head.fc_reg.weight: copying a param with shape torch.Size([320, 1024]) from checkpoint, the shape in current model is torch.Size([16, 1024]).
size mismatch for roi_head.bbox_head.fc_reg.bias: copying a param with shape torch.Size([320]) from checkpoint, the shape in current model is torch.Size([16]).
20

2022-12-06 04:06:01,102 - mmdet - INFO - Epoch [1][100/4000]	lr: 3.976e-03, eta: 1 day, 2:00:29, time: 0.468, data_time: 0.231, memory: 2014, loss_rpn_cls: 0.0406, loss_rpn_bbox: 0.0438, loss_cls: 0.5330, acc: 81.9111, loss_bbox: 0.4994, loss: 1.1167
2022-12-06 04:06:45,805 - mmdet - INFO - Epoch [1][200/4000]	lr: 7.972e-03, eta: 1 day, 1:24:10, time: 0.447, data_time: 0.214, memory: 2014, loss_rpn_cls: 0.0450, loss_rpn_bbox: 0.0485, loss_cls: 0.3068, acc: 89.5801, loss_bbox: 0.3625, loss: 0.7628
2022-12-06 04:07:31,178 - mmdet - INFO - Epoch [1][300/4000]	lr: 1.197e-02, eta: 1 day, 1:18:59, time: 0.454, data_time: 0.219, memory: 2014, loss_rpn_cls: 0.0432, loss_rpn_bbox: 0.0533, loss_cls: 0.2790, acc: 90.5059, loss_bbox: 0.3520, loss: 0.7274
2022-12-06 04:08:16,076 - mmdet - INFO - Epoch [1][400/4000]	lr: 1.596e-02, eta: 1 day, 1:12:03, time: 0.449, data_time: 0.216, memory: 2014, loss_rpn_cls: 0.0425, loss_rpn_bbox: 0.0572, loss_cls: 0.2771, acc: 90.4785, loss_bbox: 0.3541, loss: 0.7

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1000/1000, 5.3 task/s, elapsed: 188s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-12-06 04:40:07,792 - mmdet - INFO - 
+------------+------+-------+--------+-------+
| class      | gts  | dets  | recall | ap    |
+------------+------+-------+--------+-------+
| car        | 5699 | 13123 | 0.916  | 0.859 |
| truck      | 1315 | 5092  | 0.821  | 0.648 |
| bus        | 796  | 3153  | 0.878  | 0.751 |
| pedestrian | 1299 | 4750  | 0.706  | 0.495 |
+------------+------+-------+--------+-------+
| mAP        |      |       |        | 0.688 |
+------------+------+-------+--------+-------+
2022-12-06 04:40:07,885 - mmdet - INFO - Epoch(val) [1][1000]	AP50: 0.6880, mAP: 0.6885
2022-12-06 04:40:55,207 - mmdet - INFO - Epoch [2][100/4000]	lr: 2.000e-02, eta: 1 day, 1:54:43, time: 0.472, data_time: 0.237, memory: 2014, loss_rpn_cls: 0.0304, loss_rpn_bbox: 0.0473, loss_cls: 0.2078, acc: 92.5879, loss_bbox: 0.2962, loss: 0.5817
2022-12-06 04:41:40,247 - mmdet - INFO - Epoch [2][200/4000]	lr: 2.000e-02, eta: 1 day, 1:51:56, time: 0.450, data_time: 0.217, memory: 2014, loss_rp

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1000/1000, 5.3 task/s, elapsed: 188s, ETA:     0s
---------------iou_thr: 0.5---------------


...

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1000/1000, 5.3 task/s, elapsed: 190s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-12-07 08:54:59,747 - mmdet - INFO - 
+------------+------+------+--------+-------+
| class      | gts  | dets | recall | ap    |
+------------+------+------+--------+-------+
| car        | 5699 | 8352 | 0.957  | 0.941 |
| truck      | 1315 | 2782 | 0.924  | 0.875 |
| bus        | 796  | 1398 | 0.938  | 0.913 |
| pedestrian | 1299 | 2952 | 0.845  | 0.777 |
+------------+------+------+--------+-------+
| mAP        |      |      |        | 0.877 |
+------------+------+------+--------+-------+
2022-12-07 08:54:59,791 - mmdet - INFO - Epoch(val) [49][1000]	AP50: 0.8770, mAP: 0.8765
2022-12-07 08:55:46,164 - mmdet - INFO - Epoch [50][100/4000]	lr: 2.000e-04, eta: 0:31:18, time: 0.463, data_time: 0.222, memory: 2014, loss_rpn_cls: 0.0040, loss_rpn_bbox: 0.0212, loss_cls: 0.0834, acc: 96.7666, loss_bbox: 0.1564, loss: 0.2650
2022-12-07 08:56:31,040 - mmdet - INFO - Epoch [50][200/4000]	lr: 2.000e-04, eta: 0:30:30, time: 0.449, data_time: 0.212, memory: 2014, loss_rpn_cls: 0.0042, loss_r

[>>>>>>>>>>>>>>>>>>>>>>>>>>>] 1000/1000, 5.1 task/s, elapsed: 196s, ETA:     0s
---------------iou_thr: 0.5---------------


2022-12-07 09:30:57,398 - mmdet - INFO - 
+------------+------+------+--------+-------+
| class      | gts  | dets | recall | ap    |
+------------+------+------+--------+-------+
| car        | 5699 | 8362 | 0.958  | 0.941 |
| truck      | 1315 | 2702 | 0.924  | 0.875 |
| bus        | 796  | 1392 | 0.937  | 0.913 |
| pedestrian | 1299 | 2789 | 0.838  | 0.773 |
+------------+------+------+--------+-------+
| mAP        |      |      |        | 0.875 |
+------------+------+------+--------+-------+
2022-12-07 09:30:57,495 - mmdet - INFO - Epoch(val) [50][1000]	AP50: 0.8750, mAP: 0.8755


learning_rate,███████▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
momentum,▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
train/acc,▁▁▂▃▃▄▄▆▇▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇▇█▇█▇█▇█████▇██
train/loss,█▇▆▆▅▅▅▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▂▁▂▁▂▁▁▂▁▁▁▁▁
train/loss_bbox,██▇▆▆▆▆▃▃▃▂▃▃▂▃▂▂▂▂▂▂▂▂▁▂▂▁▂▁▂▂▂▂▂▂▁▁▂▂▁
train/loss_cls,█▇▆▆▅▅▅▃▂▂▂▂▂▂▂▂▂▂▂▂▂▂▂▁▂▁▁▂▁▁▁▂▁▁▁▁▁▁▁▁
train/loss_rpn_bbox,█▇▆▆▅▅▅▂▃▃▂▂▃▂▂▂▂▂▂▂▂▂▂▁▂▂▁▂▁▂▂▂▂▂▂▁▁▂▂▂
train/loss_rpn_cls,█▄▄▄▃▃▃▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
val/AP50,▁▃▃▅▅▅▆█████████████████████████████████
val/mAP,▁▃▃▅▅▅▆█████████████████████████████████
learning_rate,0.0002
